In [1]:
import ee
import geemap

# Initialize the Earth Engine API
ee.Initialize()

# Define the ROI as a polygon geometry
roi = ee.Geometry.Polygon([
    [97.0, 21.5],
    [97.0, 17.0],
    [101.5, 17.0],
    [101.5, 21.5]
])

# Convert the geometry into a FeatureCollection
roi_feature = ee.FeatureCollection([ee.Feature(roi)])

# Style the feature: red outline and no fill
styled_roi = roi_feature.style(
    color='FF0000',      # Red outline in hex
    fillColor='00000000', # Transparent fill (8-digit hex: '00000000')
    width=2              # Outline width
)

# Create a map using geemap
Map = geemap.Map()
Map.setCenter(99.25, 19.25, 7) # Center roughly on northern Thailand
Map.addLayer(styled_roi, {}, 'ROI')
Map

Map(center=[19.25, 99.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [8]:
import ee
import geemap

# Initialize the Earth Engine API
import ee

# Provide your Google Cloud project ID
project_id = 'earthengine-380405'

# Authenticate and initialize the Earth Engine API with the project ID
ee.Authenticate()
ee.Initialize(project=project_id)



# --------------------------------------------------------
# Step 1: Define Region of Interest (northern Thailand)
# --------------------------------------------------------
# roi = ee.Geometry.Polygon([
#     [97.0, 21.5],
#     [97.0, 17.0],
#     [101.5, 17.0],
#     [101.5, 21.5]
# ])

hexpolygon = ee.FeatureCollection("projects/ee-sakda/assets/ud_lst_10y");
roi = hexpolygon.geometry().bounds()

# Create a geemap Map to visualize results
Map = geemap.Map(center=[19.25, 99.25], zoom=7)

# Add the ROI boundary to the map
Map.addLayer(roi, {'color': 'black'}, 'ROI')

# --------------------------------------------------------
# Step 2: Define Time Range and Parameters
# --------------------------------------------------------
startYear = 2012
endYear = 2022  # inclusive

# MODIS LST datasets (V006)
terraDataset = 'MODIS/006/MOD11A2'  # Terra
aquaDataset = 'MODIS/006/MYD11A2'   # Aqua

# --------------------------------------------------------
# Step 3: Function to scale and mask LST
# --------------------------------------------------------
def scaleAndMaskLST(image):
    lst = image.select('LST_Day_1km').multiply(0.02)
    # Mask LST outside realistic range (0°C to 77°C in Kelvin)
    lst = lst.updateMask(lst.gt(273).And(lst.lt(350)))
    return lst.copyProperties(image, image.propertyNames())

# Visualization parameters for the Celsius LST
visParams = {
    'min': 15,     # 15°C
    'max': 40,     # 40°C
    'palette': ['blue', 'cyan', 'green', 'yellow', 'red']
}

# --------------------------------------------------------
# Step 4: Loop Over Each Year, Compute Mean LST from Terra+Aqua, and Export
# --------------------------------------------------------
for year in range(startYear, endYear + 1):
    startDate = ee.Date.fromYMD(year, 1, 1)
    endDate = ee.Date.fromYMD(year, 12, 31)

    print(year)

    # Filter Terra collection
    terraCollection = (ee.ImageCollection(terraDataset)
                       .filterDate(startDate, endDate)
                       .filterBounds(roi)
                       .map(scaleAndMaskLST))

    # Filter Aqua collection
    aquaCollection = (ee.ImageCollection(aquaDataset)
                      .filterDate(startDate, endDate)
                      .filterBounds(roi)
                      .map(scaleAndMaskLST))

    # Merge Terra and Aqua collections
    mergedCollection = terraCollection.merge(aquaCollection)

    # Compute the mean LST (Kelvin)
    meanLST = mergedCollection.mean().clip(roi)

    # Convert from Kelvin to Celsius
    meanLST_C = meanLST.subtract(273.15)

    # Add layer to the map for visualization
    Map.addLayer(meanLST_C, visParams, f'Mean LST {year} (°C)')

    # Export to Google Drive
    task = ee.batch.Export.image.toDrive(
        image=meanLST,
        description='Mean_LST_Terra_Aqua_Northern_Thailand_' + str(year),
        folder='_GEE',
        fileNamePrefix='mean_lst_modmyd_northern_th_' + str(year),
        scale=1000,
        region=roi, 
        crs='EPSG:4326'
    )
    task.start()

# Display the map
Map


2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


Map(center=[19.25, 99.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [11]:
import requests

# Get the URL of the task status
url = "https://data.tmd.go.th/api/Weather3Hours/V2/?uid=api&ukey=api12345&format=json"
aqi = "http://air4thai.pcd.go.th/forappV2/getAQI_JSON.php"
response = requests.get(aqi).json()

print(response)

{'stations': [{'stationID': '02t', 'nameTH': 'มหาวิทยาลัยราชภัฏบ้านสมเด็จเจ้าพระยา', 'nameEN': 'Bansomdejchaopraya Rajabhat University', 'areaTH': 'แขวงหิรัญรูจี เขตธนบุรี, กรุงเทพฯ', 'areaEN': 'Hiran Ruchi, Khet Thon Buri, Bangkok', 'stationType': 'GROUND', 'lat': '13.732846', 'long': '100.487662', 'forecast': [], 'AQILast': {'date': '2024-12-15', 'time': '17:00', 'PM25': {'color_id': '2', 'aqi': '49', 'value': '24.7'}, 'PM10': {'color_id': '0', 'aqi': '-1', 'value': '-1'}, 'O3': {'color_id': '0', 'aqi': '-1', 'value': '-1'}, 'CO': {'color_id': '0', 'aqi': '-1', 'value': '-1'}, 'NO2': {'color_id': '0', 'aqi': '-1', 'value': '-1'}, 'SO2': {'color_id': '0', 'aqi': '-1', 'value': '-1'}, 'AQI': {'color_id': '2', 'aqi': '49', 'param': 'PM25'}}}, {'stationID': '03t', 'nameTH': 'ริมถนนทางหลวงหมายเลข 3902 ', 'nameEN': 'Highway NO.3902 km.13 +600', 'areaTH': 'ริมถนนกาญจนาภิเษก เขตบางขุนเทียน, กรุงเทพฯ', 'areaEN': 'Kanchanaphisek Rd, Bang Khun Thian, Bangkok', 'stationType': 'GROUND', 'lat': '1

In [17]:
stations = response["stations"]
for x in stations:
    name = "ชัย"
    if name in x["nameTH"]:
        print(x)


{'stationID': '53t', 'nameTH': 'สถานีตำรวจนครบาลโชคชัย ', 'nameEN': 'Chokchai Police Station', 'areaTH': 'ริมถนนลาดพร้าว เขตวังทองหลาง, กรุงเทพฯ', 'areaEN': 'Lat Phrao Rd. Khet Wang Thonglang, Bangkok', 'stationType': 'GROUND', 'lat': '13.7954248', 'long': '100.5930298', 'forecast': [], 'AQILast': {'date': '2024-12-15', 'time': '17:00', 'PM25': {'color_id': '2', 'aqi': '46', 'value': '23.2'}, 'PM10': {'color_id': '0', 'aqi': '-1', 'value': '-1'}, 'O3': {'color_id': '0', 'aqi': '-999', 'value': '-1'}, 'CO': {'color_id': '0', 'aqi': '-1', 'value': '-1'}, 'NO2': {'color_id': '0', 'aqi': '-1', 'value': '-1'}, 'SO2': {'color_id': '0', 'aqi': '-999', 'value': '-1'}, 'AQI': {'color_id': '2', 'aqi': '46', 'param': 'PM25'}}}
{'stationID': 'o23', 'nameTH': 'สำนักงานทรัพยากรธรรมชาติและสิ่งแวดล้อมจังหวัดชัยนาท', 'nameEN': 'Natural Resources and Environment Office, Chainat', 'areaTH': 'ต.ในเมือง อ.เมือง, ชัยนาท', 'areaEN': 'Nai Mueang Subdistrict, Mueang District, Chainat', 'stationType': 'MOBILE',

In [18]:
def getByName(name="เชียง"):
    for x in stations:
        if name in x["nameTH"]:
            print(x)

In [23]:
getByName("ชัย")

{'stationID': '53t', 'nameTH': 'สถานีตำรวจนครบาลโชคชัย ', 'nameEN': 'Chokchai Police Station', 'areaTH': 'ริมถนนลาดพร้าว เขตวังทองหลาง, กรุงเทพฯ', 'areaEN': 'Lat Phrao Rd. Khet Wang Thonglang, Bangkok', 'stationType': 'GROUND', 'lat': '13.7954248', 'long': '100.5930298', 'forecast': [], 'AQILast': {'date': '2024-12-15', 'time': '17:00', 'PM25': {'color_id': '2', 'aqi': '46', 'value': '23.2'}, 'PM10': {'color_id': '0', 'aqi': '-1', 'value': '-1'}, 'O3': {'color_id': '0', 'aqi': '-999', 'value': '-1'}, 'CO': {'color_id': '0', 'aqi': '-1', 'value': '-1'}, 'NO2': {'color_id': '0', 'aqi': '-1', 'value': '-1'}, 'SO2': {'color_id': '0', 'aqi': '-999', 'value': '-1'}, 'AQI': {'color_id': '2', 'aqi': '46', 'param': 'PM25'}}}
{'stationID': 'o23', 'nameTH': 'สำนักงานทรัพยากรธรรมชาติและสิ่งแวดล้อมจังหวัดชัยนาท', 'nameEN': 'Natural Resources and Environment Office, Chainat', 'areaTH': 'ต.ในเมือง อ.เมือง, ชัยนาท', 'areaEN': 'Nai Mueang Subdistrict, Mueang District, Chainat', 'stationType': 'MOBILE',

In [34]:
def getByNameArr(name="เชียง"):
    staFilter = []
    for x in stations:
        if name in x["nameTH"]:
            staFilter.append({'nameTH': x['nameTH'], 'pm25': x['AQILast']['PM25']['value']})
    return staFilter

In [36]:
a = getByNameArr("ชัย")
# print(len(a))
print(a)

[{'nameTH': 'สถานีตำรวจนครบาลโชคชัย ', 'pm25': '23.2'}, {'nameTH': 'สำนักงานทรัพยากรธรรมชาติและสิ่งแวดล้อมจังหวัดชัยนาท', 'pm25': '13.8'}, {'nameTH': 'สนามกีฬาเทศบาลเมืองชัยภูมิ', 'pm25': '11.5'}]


In [40]:
!pip  install pandas openpyxl
import pandas as pd

def exportToExcel(arr=[]):
    df = pd.DataFrame(arr)
    df.to_excel("output.xlsx")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 1.9 MB/s eta 0:00:00a 0:00:01m


In [42]:
arr = getByNameArr("ชัย")
exportToExcel(arr)
print(arr)

[{'nameTH': 'สถานีตำรวจนครบาลโชคชัย ', 'pm25': '23.2'}, {'nameTH': 'สำนักงานทรัพยากรธรรมชาติและสิ่งแวดล้อมจังหวัดชัยนาท', 'pm25': '13.8'}, {'nameTH': 'สนามกีฬาเทศบาลเมืองชัยภูมิ', 'pm25': '11.5'}]


In [50]:
import requests
import ee
import matplotlib.pyplot as plt

# Provide your Google Cloud project ID
project_id = 'earthengine-380405'

# Authenticate and initialize the Earth Engine API with the project ID
ee.Authenticate()
ee.Initialize(project=project_id)

# Define the ROI as a polygon geometry
roi = ee.Geometry.Polygon([
    [97.0, 21.5],
    [97.0, 17.0],
    [101.5, 17.0],
    [101.5, 21.5]
])

# Define the collection
collection = ee.ImageCollection('COPERNICUS/S2') \
        .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']) \
        .filterBounds(roi) \
        .filterDate('2021-01-01', '2021-12-31')

# Function to add NDVI band
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

ndvi_collection = collection.map(add_ndvi).select('NDVI')

# การวิเคราะห์ Time Series (NDVI) โดยใช้ Pandas และ Matplotlib สร้างกราฟ Time Series
# Convert the image collection to a list
ndvi_list = ndvi_collection.toList(ndvi_collection.size())

# Create an empty list to store the data
data = []

# Loop over each image in the collection
for i in range(ndvi_list.size().getInfo()):
    image = ee.Image(ndvi_list.get(i))
    date = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd').getInfo()
    value = image.reduceRegion(
        reducer=ee.Reducer.mean(), 
        geometry=roi, 
        scale=200) \
            .values() \
            .get(0) \
            .getInfo()
    data.append({'date': date, 'ndvi': value})

# Convert the list to a Pandas DataFrame
df = pd.DataFrame(data)

# Convert the 'date' column to a datetime format
df['date'] = pd.to_datetime(df['date'])

# Set the 'date' column as the index
df = df.set_index('date')

# Plot the time series
plt.figure(figsize=(10, 5))
plt.plot(df.index, df['ndvi'], marker='o', color='b')
plt.title('NDVI Time Series')
plt.xlabel('Date')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)
plt.ylabel('NDVI')
plt.grid(True)
plt.tight_layout()

# Display the plot
plt.show()